# Zadania

In [24]:
# Two-class MNIST 
import os
os.environ['KERAS_BACKEND'] = 'theano'
from keras.datasets import mnist
import numpy as np
import random

d1 = 5
d2 = 6

(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = mnist.load_data()

X_train = (mnist_x_train.astype('float32') / 255.).reshape((len(mnist_x_train), np.prod(mnist_x_train.shape[1:])))
y_train = mnist_y_train
X_test = (mnist_x_test.astype('float32') / 255.).reshape((len(mnist_x_test), np.prod(mnist_x_test.shape[1:])))
y_test = mnist_y_test

X_train = X_train[np.logical_or(y_train == d1, y_train == d2)]
y_train = y_train[np.logical_or(y_train == d1, y_train == d2)]
y_train[y_train==d1] = 0
y_train[y_train==d2] = 1
X_test = X_test[np.logical_or(y_test == d1, y_test == d2)]
y_test = y_test[np.logical_or(y_test == d1, y_test == d2)]
y_test[y_test==d1] = 0
y_test[y_test==d2] = 1

print X_train.shape
print X_test.shape

(11339, 784)
(1850, 784)


## Ćwiczenie 1 [5 pkt]

Uzupełnij metody forward_pass oraz backward_pass w klasach ReLU, Sigmoid i Dense. Metoda forward_pass ma przyjmować batch inputów i zwracać batch outputów. Metoda backward_pass ma przyjmować batch inputów oraz batch pochodnych cząstkowych outputów i zwracać batch pochodnych cząstkowych inputów oraz wektor (**nie batch**) pochodnych cząstkowych wag. Jeśli wagi przechowujemy w macierzy dwuwymiarowej, to możemy najpierw policzyć pochodne cząstkowe w macierzy o takim samym kształcie, a następnie np. użyć .flat.

## Ćwiczenie 2 [4 pkt]

Uzupełnij metodę _forward_pass klasy Network. Metoda ta ma przyjmować batch inputów (X) i zwracać dwie rzeczy:
* inps - lista batchów inputów dla każdej warstwy w sieci (włącznie z X); te wartości będziemy używali w metodzie _backward_pass
* output - batch outputów z sieci (czyli $\mathbf{\hat y}$); output **nie** powinien być ostatnim elementem inps.

## Ćwiczenie 3 [5 pkt]

Uzupełnij metodę _backward_pass klasy Network. Zwróć uwagę, że pochodna funkcji kosztu po neuronach ostatniej warstwy jest już liczona w metodzie _fit_on_batch. Metoda ma zwracać listę layer_grads, której elementy to wektory pochodnych cząstkowych funkcji kosztu po kolejnych warstwach (zwrócone przez metodę Layer.backward_pass). Kolejność wektorów w tej liście ma być zgodna z kolejnością warstw w sieci.

## Ćwiczenie 4 [3 pkt]
Naucz sieć neuronową z jedną warstwą ukrytą i aktywacją Sigmoid na powyższych danych (dwuklasowy MNIST). Użyj MSE jako funkcji kosztu (oznacza to regresję do numeru klasy, co jest złym pomysłem, ale póki nie mamy klasy Crossentropy musi nam to wystarczyć). Użyj GD. Reportuj loss oraz accuracy.

## Ćwiczenie 5 [3 pkt]
Uzupełnić klasę Crossentropy, wzorując się na klasie MSE.

## Ćwiczenie  6 [3 pkt]
Uzupełnić klasę Momentum, wzorując się na klasie GD. Wzory można znaleźć tutaj: http://distill.pub/2017/momentum/

## Ćwiczenie 7 [3 pkt]
Naucz sieć neuronową z jedną warstwą ukrytą. Rozważ dwa przypadki: aktywację ReLU oraz Sigmoid. Czy jest sens używać ReLU jako ostatnią warstwę? Użyj Crossentropy jako funkcji kosztu. Użyj Momentum. Reportuj loss oraz accuracy.

## Ćwiczenie 8 [6 pkt]
Vanishing gradient.

Zadanie polega na zbadaniu zjawiska *vanishing gradient* w głębokich sieciach. Należy zmodyfikować kod warstwy Dense i dodać monitorowanie **normy euklidesowej** wektora delta_weights. Każdą warstwę Dense w trenowanej sieci należy monitorować oddzielnie. Po każdym wywołaniu metody fit_on_batch każdy z monitorów powinien zapamiętać nową normę. Po nauczeniu sieci dla każdej warstwy należy narysować wykres: poziomo - numer wywołania fit_on_batch, pionowo - norma delta_weights. Im niżej znajduje się warstwa Dense, tym silniej będzie zachodziło zjawisko *vanishing gradient*.

Naucz dwuwarstwową sieć z aktywacjami Sigmoid, reportując normy delta_weights. Powtórz to dla głębszej sieci (np. 6-10 warstw).

## Ćwiczenie 9 [4 pkt]
Przetestować kod z ćwiczenia 7. (dwuwarstwowa sieć) stosując inne inicjalizacje wag w warstwach Dense. Napisać własną inicjalizację wag, która sprawi, że sieć niczego się nie nauczy (init='stupid').


In [40]:
import math
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt


In [26]:
class Layer():

    def forward_pass(self, input):
        # return output
        raise NotImplementedError()

    def backward_pass(self, input, output_grad):
        # return input_grad, weight_grad
        raise NotImplementedError()

    def update_weights(self, delta_weights):
        pass

    def debug_grad(self, evaluate_loss):
        return None

In [27]:
class ReLU(Layer):
    #Ćwiczenie 1 
    def forward_pass(self, input):
        return np.asmatrix(np.vectorize(lambda x: x if x >= 0 else 0)(input))
    #Ćwiczenie 1 
    def backward_pass(self, input, output_grad):
        return  np.array([ np.multiply( output_grad[i,:],  
            np.vectorize(lambda x: 1 if x >= 0 else 0)
                (input[i,:])   ).tolist()[0] for i in xrange(input.shape[0])]), None    

In [28]:
def sigmoid(gamma):
    if gamma < 0:
        return 1. - 1./(1. + math.exp(gamma))
    else:
        return 1./(1. + math.exp(-gamma))

class Sigmoid(Layer):
    #Ćwiczenie 1 
    def forward_pass(self, input):
        return np.asmatrix(np.vectorize(sigmoid)(input))
    
    #Ćwiczenie 1 
    def backward_pass(self, input, output_grad):
        input_grad = np.array([ np.multiply( output_grad[i],  
                np.vectorize(lambda x: sigmoid(x) * sigmoid(-x) )
                    (input[i,:])   ).tolist()[0]  for i in xrange(input.shape[0])])
        weight_grad = None
        return  input_grad, weight_grad

In [29]:
class Dense(Layer):

    def __init__(self, input_size, output_size, init = 'gaussian', plot=False):
        self.plot = plot
        self.monitor = []
        input_size += 1
        if init == 'zeros':
            self.weights = np.zeros((input_size, output_size))
        elif init == 'gaussian':
            np.random.seed(1)
            self.weights = np.random.normal(
                0.,
                2. / (input_size + output_size),
                (input_size, output_size)
            )
        elif init == 'aaa':
            np.random.seed(1)
            self.weights = np.random.choice([0.001, 1000.01],  (input_size, output_size)  )
        else:
            raise NotImplementedError()
        self.weights = np.asmatrix(self.weights)

    #Ćwiczenie 1 
    def forward_pass(self, input):
        input = np.append(input, np.array([[1] for i in input]), axis=1)
        return np.dot(input, self.weights)
    
    #Ćwiczenie 1 
    def backward_pass(self, input, output_grad):
        N = len(input)
        input = np.append(input, np.array([[1] for i in input]), axis=1)
        n = input.shape[1] 
        m = output_grad.shape[1]
        input_grad = np.dot(output_grad, (self.weights[:-1,:]).T)
        weight_grad=  np.array( [ [  np.sum( np.multiply(output_grad[:,b], input[:,a]) ) 
                                   for b in xrange(m)]  for a in xrange(n) ] )  
        
        return input_grad, weight_grad
        
    def update_weights(self, delta_weights):
        self.weights += delta_weights
        self.monitor.append(np.linalg.norm(delta_weights))

In [30]:
# Optimizers
class Optimizer():

    def calculate_deltas(self, grad):
        raise NotImplementedError()

class GD(Optimizer):

    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def calculate_deltas(self, grad):
        return -self.learning_rate * grad      

In [31]:
#Ćwiczenie 6
class Momentum(Optimizer):

    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta
        self.remember = 0
        self.ready = False
    def calculate_deltas(self, grad):
        #if self.ready == False:
        #    self.remember = grad
        #    self.ready = True
        
        self.remember = self.beta * self.remember + grad
        return -self.alpha * self.remember

In [32]:
class Loss():

    def forward_pass(self, y, t):
        # return cost
        raise NotImplementedError()

    def backward_pass(self, y, t):
        # return y_grad
        raise NotImplementedError()

class MSE(Loss):

    def forward_pass(self, y, t):
        return np.average(0.5 * np.square(y - t))

    def backward_pass(self, y, t):
        return (y - t) / y.size

#Ćwiczenie 5
class Crossentropy(Loss):

    def forward_pass(self, y, t):
        return np.average(-1.0*np.multiply(t, np.log(y))
                          -np.multiply(1.0 - t, np.log( 1.0 - y )) )
        
    def backward_pass(self, y, t):
        return ( -1.0*np.divide(t,y) + np.divide((1.0-t),(1.0-y)) )/ y.size


In [52]:
class Network():
    def __init__(self, loss, optimizer, metrics = []):
        self.layers = []
        self.loss = loss
        self.optimizer = optimizer
        self.metrics = metrics

    def add(self, layer):
        self.layers.append(layer)

    def fit(self, X, t, epochs, batch_size=256, print_stats=False,plot=False):
        X = np.array(X)
        t = np.array(t)
        X = X.reshape(len(X), -1)
        t = t.reshape(len(t), -1)
        if X.shape[0] != t.shape[0]:
            raise ValueError("Array sizes don't match")

        times = 0
        norms = []
        
        for epoch in range(epochs):
            if print_stats:
                print("Epoch %d" % (epoch+1))
                print("    -> batch size: %d" % batch_size)
            rng_state = np.random.get_state()
            np.random.shuffle(X)
            np.random.set_state(rng_state)
            np.random.shuffle(t)
            pos = 0
            while pos < len(X):
                batch_X = X[pos:pos+batch_size]
                batch_t = t[pos:pos+batch_size]
                self._fit_on_batch(batch_X, batch_t)
                times+=1
                #norms = norms.append()
        
        
                pos += batch_size
            if print_stats:
                _, y = self._forward_pass(X)
                l = self.loss.forward_pass(y, t)
                print("    -> loss: %f" % l)
                for m in self.metrics:
                    print("    -> %s: %f" % (m.__name__, m(y, t)))
        if plot:
            nDenses = sum([layer.__class__.__name__ == "Dense" for layer in self.layers])
            maximaxi = np.max(np.max([layer.monitor  for layer in self.layers if (layer.__class__.__name__ == "Dense") and layer != None ]))
            plt.clf()
            plt.cla()
            plt.close()
            tmp =4 * int(math.ceil(nDenses / 5.0))
            plt.figure(figsize=(20,tmp), dpi=80)
            plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
            iteri = 1
            for i, layer in enumerate( self.layers):
                if not layer.__class__.__name__ == "Dense":
                    continue
                if layer.plot:
                    plt.subplot(int(math.ceil(nDenses / 5.0)) ,5,iteri)
                    iteri+=1
                    plt.title("Layer "  +str(i))
                    plt.ylim([0, maximaxi])
                    plt.yscale('symlog')
                    plt.bar( [x for x in range(len(layer.monitor))], layer.monitor)
            plt.show()
        
        
    def predict(self, X):
        inps, out = self._forward_pass(X)
        return out

    def _fit_on_batch(self, batch_X, batch_t):
        inps, out = self._forward_pass(batch_X)
        layer_grads = self._backward_pass(
            inps,
            self.loss.backward_pass(out, batch_t)
        )

        grad = self._join(layer_grads)
        
        deltas = self.optimizer.calculate_deltas(grad)
        for l, d in zip(self.layers, deltas):
            if not d is None:
                l.update_weights(d)
        

    def _join(self, grads):
        return np.array([g for g in grads if not g is None])

    def _split(self, grads, layer_grads):
        out = []
        start = 0
        for l in layer_grads:
            if l is None:
                out.append(None)
            else:
                out.append(grads[start:start+len(l)])
                start += len(l)
        return out
    
    #Ćwiczenie 2
    def _forward_pass(self, X):
        inps = []
        output = None
        inps.append(X)
        for layer in self.layers:
            inps.append(layer.forward_pass(inps[-1]))
            
        output = inps[-1]
        inps.pop()
        return inps, output
    #Ćwiczenie 3
    def _backward_pass(self, inps, grad):
        n = len(self.layers)
        layer_grads = [None for i in xrange(n)]
        weight_grad = [None for i in xrange(n)]
        layer_grads[n-1] = grad

        for i in xrange(n-1, 0,-1):
            input_grad, weights_grad = self.layers[i-1].backward_pass((inps[i-1]), layer_grads[i]  ) 
            weight_grad[i-1] =  weights_grad
            layer_grads[i-1] =  input_grad
        return weight_grad

    def _debug_grads(self, X, t):
        layer_grads = []
        for l in self.layers:
            g = l.debug_grad(
                lambda: self.loss.forward_pass(self._forward_pass(X)[1], t)
            )
            if not g is None:
                g = np.array(np.array(g).flat)
            layer_grads.append(g)
        return layer_grads

In [53]:
# Ćwiczenie 4
from sklearn.metrics import accuracy_score
def metric(y,t):
    return accuracy_score(np.around(y.flat), t)

network = Network(loss=MSE(), optimizer=GD(learning_rate=0.05), metrics=[metric])
network.add(Dense(784,8))
network.add(Dense(8,1)) #jedna warstwa ukryta
network.add(Sigmoid())
network.fit(X_train[:1000],y_train[:1000], epochs= 2, print_stats=True)
print metric(network.predict(X_test), y_test)

Epoch 1
    -> batch size: 256
    -> loss: 0.230000
    -> metric: 0.540000
Epoch 2
    -> batch size: 256
    -> loss: 0.270000
    -> metric: 0.460000
0.482162162162


In [55]:
# Ćwiczenie 7:1
network1 = Network(loss=Crossentropy(), optimizer=Momentum(alpha=0.01,beta=0.05), metrics=[metric])
network1.add(Dense(784,32))
network1.add(Dense(32,1))
network1.add(Sigmoid())
network1.fit(X_train[:1000],y_train[:1000], epochs= 2, print_stats=True)
print metric(network1.predict(X_test), y_test)

Epoch 1
    -> batch size: 256


KeyboardInterrupt: 

In [ ]:
# Ćwiczenie 7:2
network2 = Network(loss=Crossentropy(), optimizer=Momentum(alpha=0.01,beta=0.05), metrics=[metric])
network2.add(Dense(784,32))
network2.add(Dense(32,1))
network2.add(ReLU())
network2.fit(X_train[:1000],y_train[:1000], epochs= 2, print_stats=True)
print metric(network2.predict(X_test), y_test)

Ćwiczenie 7:
Nie ma sensu ReLU jako ostatnia warstwa, ponieważ rozważany problem to problem klasyfikacji binarnej. Zwracane wartości to powinno być 0 lub 1, ew "prawdopodobieństwo". ReLU wyrzuca cokolwiek dodatniego. Być może dużego. 


In [ ]:
# Ćwiczenie 8:1
network3 = Network(loss=Crossentropy(), optimizer=Momentum(alpha=0.01,beta=0.05), metrics=[metric])
network3.add(Dense(784,32,plot=True))
network3.add(Dense(32,8,plot=True))
network3.add(Sigmoid())
network3.add(Dense(8,1,plot=True))
network3.add(Sigmoid())
network3.fit(X_train[:100],y_train[:100], epochs= 3, print_stats=True,plot=True)
print metric(network3.predict(X_test), y_test)

In [ ]:
# Ćwiczenie 8:2
network4 = Network(loss=Crossentropy(), optimizer=Momentum(alpha=0.01,beta=0.05), metrics=[metric])
network4.add(Dense(784,32,plot=True))
network4.add(Dense(32,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,1,plot=True))
network4.add(Sigmoid())
network4.fit(X_train[:400],y_train[:400], epochs= 3, print_stats=True,plot=True)
print metric(network4.predict(X_test), y_test)

In [ ]:
#Ćwiczenie 9
network5 = Network(loss=MSE(), optimizer=Momentum(alpha=0.01,beta=0.05), metrics=[metric])
network5.add(Dense(784,32,init='aaa'))
network5.add(Dense(32,8,init='aaa'))
network5.add(Sigmoid())
network5.add(Dense(8,1,init='aaa'))
network5.add(Sigmoid())
network5.fit(X_train[:100],y_train[:100], epochs= 3, print_stats=True)
print metric(network5.predict(X_test), y_test)


In [44]:
from sklearn.model_selection import train_test_split

adata_x = np.array([ [x] for x in np.arange(0., 2.0 * math.pi, 0.01) ])
adata_y = np.vectorize(lambda x: math.sin(x))(adata_x)
adata_x_train, adata_x_test, adata_y_train, adata_y_test = train_test_split(adata_x, adata_y, test_size=0.33, random_state=43)


In [ ]:
from sklearn.metrics import mean_absolute_error

network8 = Network(loss=Crossentropy(), optimizer=GD(learning_rate=0.05), metrics=[mean_absolute_error])
network8.add(Dense(1,81,plot=True))
network8.add(Dense(81,1,plot=True))
network8.add(ReLU())
network8.fit(adata_x_train,adata_y_train, epochs= 5, print_stats=True)
print mean_absolute_error(network8.predict(adata_x_test), adata_y_test)

plt.plot(adata_x, adata_y)
plt.plot(adata_x, network8.predict(adata_x))

## Ćwiczenie 1

In [23]:
np.random.seed(43)
d1 = Dense(input_size=4, output_size=3, init="gaussian")
d2 = Dense(input_size=4, output_size=3, init="zeros")
r = ReLU()
s = Sigmoid()
inp = np.random.random(4).reshape((1,-1)) - 0.5
out_grad_4 = np.random.random(4).reshape((1,-1)) - 0.5
out_grad_3 = np.random.random(3).reshape((1,-1)) - 0.5

print "inp:"
print inp.shape
print inp
print "out_grad_4:"
print out_grad_4.shape
print out_grad_4
print "out_grad_3:"
print out_grad_3.shape
print out_grad_3

print "Testing d1..."
print "d1.forward_pass(inp):"
t = d1.forward_pass(inp)
print t.shape
print t
print "d1.backward_pass(inp, out_grad_3):"
t = d1.backward_pass(inp, out_grad_3)
print t[0].shape
print t[0]
print t[1].shape
print t[1]

print "Testing d2..."
print "d2.forward_pass(inp):"
t = d2.forward_pass(inp)
print t.shape
print t
print "d2.backward_pass(inp, out_grad_3):"
t = d2.backward_pass(inp, out_grad_3)
print t[0].shape
print t[0]
print t[1].shape
print t[1]

print "Testing r..."
print "r.forward_pass(inp):"
t = r.forward_pass(inp)
print t.shape
print t
print "r.backward_pass(inp, out_grad_4):"
t = r.backward_pass(inp, out_grad_4)
print t[0].shape
print t[0]
print t[1]

print "Testing s..."
print "s.forward_pass(inp):"
t = s.forward_pass(inp)
print t.shape
print t
print "s.backward_pass(inp, out_grad_4):"
t = s.backward_pass(inp, out_grad_4)
print t[0].shape
print t[0]
print t[1]

inp:
(1, 4)
[[ 0.37638915  0.39460666 -0.41495579 -0.46094522]]
out_grad_4:
(1, 4)
[[-0.33016958  0.3781425  -0.40165317 -0.07889237]]
out_grad_3:
(1, 3)
[[ 0.45788953  0.03316528  0.19187711]]
Testing d1...
d1.forward_pass(inp):
(1, 3)
[[-0.18587636 -0.15772542  0.21099843]]
d1.backward_pass(inp, out_grad_3):
(1, 4)
[[ 0.1555344  -0.22605305  0.20872542 -0.11524668]]
(5, 3)
[[ 0.17234465  0.01248305  0.07222046]
 [ 0.18068626  0.01308724  0.07571599]
 [-0.19000391 -0.01376213 -0.07962052]
 [-0.21106199 -0.01528738 -0.08844484]
 [ 0.45788953  0.03316528  0.19187711]]
Testing d2...
d2.forward_pass(inp):
(1, 3)
[[ 0.  0.  0.]]
d2.backward_pass(inp, out_grad_3):
(1, 4)
[[ 0.  0.  0.  0.]]
(5, 3)
[[ 0.17234465  0.01248305  0.07222046]
 [ 0.18068626  0.01308724  0.07571599]
 [-0.19000391 -0.01376213 -0.07962052]
 [-0.21106199 -0.01528738 -0.08844484]
 [ 0.45788953  0.03316528  0.19187711]]
Testing r...
r.forward_pass(inp):
(1, 4)
[[ 0.37638915  0.39460666  0.          0.        ]]
r.backwar

## Ćwiczenie 2

In [20]:
np.random.seed(43)
n = Network(loss=MSE(), optimizer=GD(learning_rate=0.001), metrics=[])
n.add(Dense(input_size=4, output_size=3, init="gaussian"))
n.add(ReLU())
n.add(Dense(input_size=3, output_size=4, init="gaussian"))
n.add(ReLU())
n.add(Dense(input_size=4, output_size=1, init="gaussian"))
n.add(Sigmoid())
inp = np.random.random((2,4)) - 0.5
inps, out = n._forward_pass(inp)

print "inp:"
print inp
for i, inp in enumerate(inps):
    print "inps[" + str(i) + "]:"
    print inp
print "out:"
print out

inp:
[[-0.08080549  0.1852195  -0.29554775  0.37811744]
 [-0.47261241  0.17046751 -0.0826952   0.05868983]]
inps[0]:
[[-0.08080549  0.1852195  -0.29554775  0.37811744]
 [-0.47261241  0.17046751 -0.0826952   0.05868983]]
inps[1]:
[[-0.3155936   0.1508727  -0.03077691]
 [-0.35798308  0.05033805  0.21094002]]
inps[2]:
[[0 0 0]
 [0 0 0]]
inps[3]:
[[-0.0806043  -0.09601359  0.28344236 -0.27497282]
 [-0.0806043  -0.09601359  0.28344236 -0.27497282]]
inps[4]:
[[0 0 0 0]
 [0 0 0 0]]
inps[5]:
[[ 0.28846921]
 [ 0.28846921]]
out:
[[ 0.57162133]
 [ 0.57162133]]


## Ćwiczenie 3

In [18]:
np.random.seed(43)
n = Network(loss=MSE(), optimizer=GD(learning_rate=0.001), metrics=[])
n.add(Dense(input_size=4, output_size=3, init="gaussian"))
n.add(ReLU())
n.add(Dense(input_size=3, output_size=4, init="gaussian"))
n.add(ReLU())
n.add(Dense(input_size=4, output_size=1, init="gaussian"))
n.add(Sigmoid())

inp = np.random.random((2,4)) - 0.5
target = inp[:,0:1]
inps, out = n._forward_pass(inp)
grad = n.loss.backward_pass(out, target)
layer_grads = n._backward_pass(inps, grad)

print "inp:"
print inp
print "target:"
print target
for i, inp in enumerate(inps):
    print "inps[" + str(i) + "]:"
    print inp
print "out:"
print out
print "grad:"
print grad
for i, grad in enumerate(layer_grads):
    print "layer_grads[" + str(i) + "]:"
    print grad

inp:
[[-0.08080549  0.1852195  -0.29554775  0.37811744]
 [-0.47261241  0.17046751 -0.0826952   0.05868983]]
target:
[[-0.08080549]
 [-0.47261241]]
inps[0]:
[[-0.08080549  0.1852195  -0.29554775  0.37811744]
 [-0.47261241  0.17046751 -0.0826952   0.05868983]]
inps[1]:
[[-0.3155936   0.1508727  -0.03077691]
 [-0.35798308  0.05033805  0.21094002]]
inps[2]:
[[0 0 0]
 [0 0 0]]
inps[3]:
[[-0.0806043  -0.09601359  0.28344236 -0.27497282]
 [-0.0806043  -0.09601359  0.28344236 -0.27497282]]
inps[4]:
[[0 0 0 0]
 [0 0 0 0]]
inps[5]:
[[ 0.28846921]
 [ 0.28846921]]
out:
[[ 0.57162133]
 [ 0.57162133]]
grad:
[[ 0.32621341]
 [ 0.52211687]]
layer_grads[0]:
[[ 0.          0.02097462  0.01587984]
 [ 0.         -0.01147535 -0.00572773]
 [ 0.          0.01071991  0.00277857]
 [ 0.         -0.01182592 -0.00197199]
 [ 0.         -0.06514896 -0.03360014]]
layer_grads[1]:
None
layer_grads[2]:
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.

## Ćwiczenie 5

In [17]:
np.random.seed(43)
ce = Crossentropy()
y = np.random.random(10)
t = np.random.random(10)

print "y:"
print y
print "t:"
print t
print "ce.forward_pass(y,t):"
print ce.forward_pass(y,t)
print "ce.backward_pass(y,t):"
print ce.backward_pass(y,t)

y:
[ 0.11505457  0.60906654  0.13339096  0.24058962  0.32713906  0.85913749
  0.66609021  0.54116221  0.02901382  0.7337483 ]
t:
[ 0.39495002  0.80204712  0.25442113  0.05688494  0.86664864  0.221029
  0.40498945  0.31609647  0.0766627   0.84322469]
ce.forward_pass(y,t):
0.736415962327
ce.backward_pass(y,t):
[-0.27490047 -0.08104869 -0.10469935  0.10054647 -0.24509895  0.5272741
  0.11739401  0.0906406  -0.16913545 -0.05603779]


## Ćwiczenie 6

In [22]:
np.random.seed(43)
opt = Momentum(alpha=0.02, beta=0.99)
grad1 = np.random.random(5)
grad2 = np.random.random(5)
grad3 = np.random.random(5)
opt.calculate_deltas(grad1)
opt.calculate_deltas(grad2)
opt.calculate_deltas(grad3)

print "grad1:"
print grad1
print "grad2:"
print grad2
print "grad3:"
print grad3

print "opt.calculate_deltas(grad1):"
print opt.calculate_deltas(grad1)
print "opt.calculate_deltas(grad2):"
print opt.calculate_deltas(grad2)
print "opt.calculate_deltas(grad3):"
print opt.calculate_deltas(grad3)

grad1:
[ 0.11505457  0.60906654  0.13339096  0.24058962  0.32713906]
grad2:
[ 0.85913749  0.66609021  0.54116221  0.02901382  0.7337483 ]
grad3:
[ 0.39495002  0.80204712  0.25442113  0.05688494  0.86664864]
opt.calculate_deltas(grad1):
[-0.02919466 -0.0529381  -0.0209018  -0.01117572 -0.04443381]
opt.calculate_deltas(grad2):
[-0.04608546 -0.06573052 -0.03151603 -0.01164424 -0.05866444]
opt.calculate_deltas(grad3):
[-0.05352361 -0.08111416 -0.03628929 -0.0126655  -0.07541077]
